<a href="https://colab.research.google.com/github/HarlodCheng/tf_colab/blob/master/minst1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#载入数据集
#mnist=input_data.read_data_sets("content/MNIST",one_hot=TRUE)
INPUT_NODE=784
OUTPUT_NODE=10
#与MNIST数据相关

LAYER1_NODE=500#隐藏层结点
BATCH_SIZE=100

LEARNING_RATE_BASE=0.8#学习率
LEARNING_RATE_DECAY=0.99
REGULARIZATION_RATE=0.0001
TRAIN_STEPS=30000#CISHU
MOVING_AVERAGE_DECAY=0.99

def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
  if avg_class==None:
    layer1=tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
    return tf.matmul(layer1,weights2)+biases2
  else:
      layer1=tf.nn.relu(
          tf.matmul(input_tensor,avg_class.average(weights1))+
          avg_class.average(biases1)
      )
      return tf.matmul(layer1,avg_class.average(weights2))+avg_class.average(biases2)

def train(mnist):
    x=tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')

    #生成隐藏层的参数
    weights1=tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1=tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))

    #输出层的参数w2
    weights2=tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2=tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))

    y=inference(x,None,weights1,biases1,weights2,biases2)

    global_step=tf.Variable(0,trainable=False)#将训练的轮数设定为不可训练

    variable_averages=tf.train.ExponentiaMovingAverage(MOVING_AVERAGE_DECAY,global_step)

    variable_averages_op=variable_averages.apply(tf.trainable_variabled())

    average_y=inference(x,variable_averages,weights1,biases1,weights2,biases2)

    cross_entropy=tf.nn.sparse_sofemax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)

    regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)

    reglarization=regularizer(weights1)+regularizer(weights2)
    loss=cross_entropy_mean+reglarization

    #设置指数衰减的学习率
    learning_rate=tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_example/BATCH_SIZE,
        LEARNING_RATE_DECAY
    )

train_step=tf.train.GradientDescentOptimizer(learning_rate)\
.minimize(loss,global_step=global_step)

with tf.control_dependencies([train_step,variable_averages_op]):
    train_op=tf.no_op(name='train')

    correct_prediction=tf.equal(tf.argmax(average_y,1),tf.argmax(average_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    validate_feed={x:mnist.validation.images,y_:mnist.validation.labels}
    test_feed={x:mnist.tset.images,y_:mnist.test.labels}

    #迭代训练神经网络
    for i in range(TRAIN_STEPS):
        if i%1000==0:
            validate_acc=sess.run(accuracy,feed_dict=validate_feed)
            print("after %d training steps,validation accuracy using average modle is %g"%(i,validate_acc))
        xs,ys=mnist.train.next_batch(BATCH_SIZE)
        sess.run(train_op,feed_dict={x:xs,y_:ys})

    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("after %d training steps,test accurracy using average model is %g"%(TRAIN_STEPS,test_acc))


#main api
def main(argv=None):
  mnist=input_data.read_data_sets("/tmp/data",one_hot=True)
if __name__=='__main__':
    tf.app.run()


NameError: ignored

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# MNIST数据存放的路径
file = "./MNIST"

# 导入数据
mnist = input_data.read_data_sets(file, one_hot=True)

# 模型的输入和输出
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# 模型的权重和偏移量
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 创建Session
sess = tf.InteractiveSession()
# 初始化权重变量
sess.run(tf.global_variables_initializer())

y = tf.nn.softmax(tf.matmul(x, W) + b)

# 交叉熵
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# 训练
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1500):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# 测试
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels}))


Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.9162


In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# MNIST数据存放的路径
file = "MNIST"

# 导入数据
mnist = input_data.read_data_sets(file, one_hot=True)


Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
